In [1]:
import pandas
import datetime
import geomag
import numpy

In [2]:
# some constants
cutOffLosVel = 10.

In [3]:
#### In this block we load Velocity data ####
#### In this block we load Velocity data ####
#### In this block we load Velocity data ####
# need the formatted velocity file to read data
outFormattedFile = "/home/bharat/Documents/code/new-vel-data/veldata/saps-vels-20130629.txt"#"../data/new-test-vels-north.txt"
# os.system( 'cat ' + rawVelFileName + ' | tr -s "  " > ' + outFormattedFile )
inpColNames = [ "dateStr", "timeStr", "beam", "range", \
          "geoAzm", "azimCalcMag", "vLos", "spwdth", "pwr", "MLAT", "MLON", "MLT",  \
          "GLAT", "GLON", "radId", "radCode"]
velsDataDF = pandas.read_csv(outFormattedFile, delim_whitespace=True,\
                             header=None, names=inpColNames)
# add a datetime col
velsDataDF["date"] = pandas.to_datetime( \
                        velsDataDF['dateStr'].astype(str) + "-" +\
                        velsDataDF['timeStr'].astype(str), format='%Y%m%d-%H%M')
# for some reason MLAT is a str type, convert it to float
velsDataDF["MLAT"] = velsDataDF["MLAT"].astype(float)
# Also get a normMLT for plotting
velsDataDF['normMLT'] = [x-24 if x >= 12 else x for x in velsDataDF['MLT']]
velsDataDF.head()
#### In this block we load SAPS Velocity data ####
#### In this block we load SAPS Velocity data ####
#### In this block we load SAPS Velocity data ####

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT
0,20130629,30,0,2,-1.36,-6.1706,5.3534,65.3794,18.4318,51.3087,0.4091,19.8614,39.9141,-75.7461,32,wal,2013-06-29 03:00:00,-4.1386
1,20130629,30,0,3,-1.36,-4.6003,8.3669,70.3744,18.4318,51.7154,0.4840,19.8664,40.3543,-75.7203,32,wal,2013-06-29 03:00:00,-4.1336
2,20130629,30,0,4,-1.36,-3.5279,10.0253,53.7272,17.9533,52.1130,0.5431,19.8703,40.7831,-75.7052,32,wal,2013-06-29 03:00:00,-4.1297
3,20130629,30,0,5,-1.36,-2.7499,20.5631,0.0100,16.2314,52.5053,0.5924,19.8736,41.2050,-75.6965,32,wal,2013-06-29 03:00:00,-4.1264
4,20130629,30,0,6,-1.36,-2.1616,24.0958,0.0100,16.2314,52.8943,0.6351,19.8764,41.6225,-75.6922,32,wal,2013-06-29 03:00:00,-4.1236


In [4]:
# convert azimuth to magn coords
gm = geomag.geomag.GeoMag()
def convert_to_mag_azm(row):
#     print type(row["date"].to_datetime()),row["date"].to_datetime()
    mg = gm.GeoMag(row['GLAT'], \
                   row['GLON'], h=300., time=row['date'].date())
    azm_mag = (round(row['geoAzm'] - mg.dec,2))
    return azm_mag


velsDataDF["magAzm"] = -999.#velsDataDF.apply( convert_to_mag_azm, axis=1 )
velsDataDF.head()

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,magAzm
0,20130629,30,0,2,-1.36,-6.1706,5.3534,65.3794,18.4318,51.3087,0.4091,19.8614,39.9141,-75.7461,32,wal,2013-06-29 03:00:00,-4.1386,-999.0
1,20130629,30,0,3,-1.36,-4.6003,8.3669,70.3744,18.4318,51.7154,0.4840,19.8664,40.3543,-75.7203,32,wal,2013-06-29 03:00:00,-4.1336,-999.0
2,20130629,30,0,4,-1.36,-3.5279,10.0253,53.7272,17.9533,52.1130,0.5431,19.8703,40.7831,-75.7052,32,wal,2013-06-29 03:00:00,-4.1297,-999.0
3,20130629,30,0,5,-1.36,-2.7499,20.5631,0.0100,16.2314,52.5053,0.5924,19.8736,41.2050,-75.6965,32,wal,2013-06-29 03:00:00,-4.1264,-999.0
4,20130629,30,0,6,-1.36,-2.1616,24.0958,0.0100,16.2314,52.8943,0.6351,19.8764,41.6225,-75.6922,32,wal,2013-06-29 03:00:00,-4.1236,-999.0


In [5]:
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
# remove velocies whose magnitude is less than 200 m/s
velsDataDF = velsDataDF[ abs(velsDataDF["vLos"]) >= cutOffLosVel ]
# SAPS(westward) vLos are positive for positive azimuths and vice versa.
# filter the others out
velsDataDF = velsDataDF[ velsDataDF["azimCalcMag"]/velsDataDF["vLos"] > 0. ].reset_index(drop=True)
velsDataDF.head()
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,magAzm
0,20130629,30,0,42,-1.36,0.3484,12.1962,20.2269,8.0493,66.2521,1.0556,19.9045,55.8318,-76.2258,32,wal,2013-06-29 03:00:00,-4.0955,-999.0
1,20130629,30,0,43,-1.36,0.3274,12.1962,20.2269,8.9077,66.6306,1.0490,19.9040,56.2367,-76.2538,32,wal,2013-06-29 03:00:00,-4.0960,-999.0
2,20130629,30,0,44,-1.36,0.3021,10.5115,0.0100,8.0493,67.0089,1.0399,19.9034,56.6417,-76.2835,32,wal,2013-06-29 03:00:00,-4.0966,-999.0
3,20130629,30,1,5,1.88,0.7345,22.2277,0.0100,19.4605,52.4810,0.9744,19.8991,41.2093,-75.4230,32,wal,2013-06-29 03:00:00,-4.1009,-999.0
4,20130629,30,1,6,1.88,1.3978,24.0958,0.0100,20.3193,52.8658,1.0609,19.9048,41.6258,-75.3877,32,wal,2013-06-29 03:00:00,-4.0952,-999.0


In [6]:
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
datFileName = "../data/processedSaps.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ', dtype={'dateStr':'str', 'time': 'str'})

sapsDataDF["date"] = pandas.to_datetime( \
                        sapsDataDF['dateStr'] + "-" +\
                        sapsDataDF['time'], format='%Y%m%d-%H%M')
sapsDataDF.head()
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####

,dateStr,sapsLat,sapsMLT,sapsVel,radId,poesLat,poesMLT,dst_date,dst_index,time,date
0,20110107,56.5,17.7543,308.2077,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
1,20110107,55.5,18.0147,224.1588,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
2,20110107,56.5,17.8749,307.4328,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
3,20110107,55.5,18.1324,222.4787,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
4,20110107,56.5,17.9955,305.4201,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07


In [7]:
velsDataDF["date"].unique()

array(['2013-06-29T03:00:00.000000000', '2013-06-29T03:02:00.000000000',
       '2013-06-29T03:04:00.000000000', '2013-06-29T03:06:00.000000000',
       '2013-06-29T03:08:00.000000000', '2013-06-29T04:00:00.000000000',
       '2013-06-29T04:02:00.000000000', '2013-06-29T04:04:00.000000000',
       '2013-06-29T04:06:00.000000000', '2013-06-29T04:08:00.000000000',
       '2013-06-29T05:00:00.000000000', '2013-06-29T05:02:00.000000000',
       '2013-06-29T05:04:00.000000000', '2013-06-29T05:06:00.000000000',
       '2013-06-29T05:08:00.000000000', '2013-06-29T10:00:00.000000000',
       '2013-06-29T10:02:00.000000000', '2013-06-29T10:04:00.000000000',
       '2013-06-29T10:06:00.000000000', '2013-06-29T10:08:00.000000000',
       '2013-06-29T11:00:00.000000000', '2013-06-29T11:02:00.000000000',
       '2013-06-29T11:04:00.000000000', '2013-06-29T11:06:00.000000000',
       '2013-06-29T11:08:00.000000000', '2013-06-29T12:00:00.000000000',
       '2013-06-29T12:02:00.000000000', '2013-06-29

In [8]:

#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
# we choose 0840 UT since that time was analyzed in the paper
timeSlctd = datetime.datetime( 2013, 6, 29, 2, 50 )
velAnlysDF = velsDataDF[ velsDataDF["date"] == timeSlctd ].reset_index(drop=True)
sapsSelPrdDF = sapsDataDF[  ( sapsDataDF["date"] - timeSlctd < numpy.timedelta64(40,'m') )\
           & ( sapsDataDF["date"] - timeSlctd > numpy.timedelta64(0,'m') )  ].reset_index(drop=True)
# Now check if there is more than one time period in the selected interval
# ideally there shouldn't be, but if we find one we should do something
# about it (like take an average)...for now its undecided.!
if len( sapsSelPrdDF["time"].unique() ) > 1:
    print "There is more than one time period selected!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
poesBndDF = sapsSelPrdDF[ ["poesMLT", "poesLat"] \
                    ].drop_duplicates().reset_index(drop=True)
# Have a normalized MLT for ease of comparison
poesBndDF['normMLT'] = [x-24 if x >= 12 else x for x in poesBndDF['poesMLT']]
poesBndDF.head()
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####

,poesMLT,poesLat,normMLT
0,16.0,107.0526,-8.0
1,17.0,112.7113,-7.0
2,18.0,116.4681,-6.0
3,19.0,118.0670,-5.0
4,21.0,114.5099,-3.0


In [9]:
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
# Merge POES boundary DF with the vels DF
velAnlysDF["normMLTRound"] = velAnlysDF["normMLT"].astype(int)
print "shape before join--->", velAnlysDF.shape
velAnlysDF = pandas.merge( velAnlysDF, poesBndDF, left_on="normMLTRound", right_on="normMLT", how="inner" )
print "shape after join--->", velAnlysDF.shape
# Filter out velocties above the POES boundary
velAnlysDF = velAnlysDF[ velAnlysDF["MLAT"] < velAnlysDF["poesLat"]\
                       ].reset_index(drop=True).drop_duplicates()
print "shape after filtering by boundary--->", velAnlysDF.shape
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####

shape before join---> (1564, 20)
shape after join---> (946, 23)
shape after filtering by boundary---> (946, 23)


In [10]:
# save the data in a csv for lshell fitting
outSelCols = [ "beam", "range", "geoAzm", "azimCalcMag", "magAzm", "vLos"\
           , "MLAT", "MLT", "MLON", "GLAT", "GLON", "radId"\
           , "radCode","normMLT_x", "normMLTRound", "spwdth", "pwr" ]
velAnlysDF.to_csv( "../data/test-losVels.txt", columns=outSelCols, index=False )